#### Import the libraries

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/MyDrive/')
# Python method used to change the current working directory to the specified path.
path = "/content/drive/MyDrive/Artigos_IC/"
os.chdir(path)
import shutil
shutil.copy('/content/drive/MyDrive/Artigos_IC/out_clean.csv', '/content/')

Mounted at /content/drive


'/content/out_clean.csv'

In [2]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast

In [2]:
from pycaret.classification import *
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv('/content/out_clean.csv', index_col=0)
#df_type = df['Type']
df.shape

(13489, 70)

#### Remove duplicates

In [4]:
duplicate_rows = df[df.duplicated()]
len(duplicate_rows)

2621

In [5]:
df.columns.values

array(['smell', 'severity', 'longmethod_label', 'featureenvy_label',
       'dataclass_label', 'blob_label', 'agreement_has_smell',
       'agreement_experts_has_smell', 'type', 'ck_class_cbo',
       'ck_class_cbomodified', 'ck_class_fanin', 'ck_class_fanout',
       'ck_class_wmc', 'ck_class_dit', 'ck_class_rfc', 'ck_class_lcom',
       'ck_class_totalmethodsqty', 'ck_class_publicmethodsqty',
       'ck_class_visiblemethodsqty', 'ck_class_totalfieldsqty',
       'ck_class_privatefieldsqty', 'ck_class_finalfieldsqty',
       'ck_class_loc', 'ck_class_returnqty', 'ck_class_comparisonsqty',
       'ck_class_stringliteralsqty', 'ck_class_numbersqty',
       'ck_class_assignmentsqty', 'ck_class_variablesqty',
       'ck_class_maxnestedblocksqty', 'ck_class_uniquewordsqty',
       'ck_class_modifiers', 'ck_method_cbo', 'ck_method_wmc',
       'ck_method_rfc', 'ck_method_loc', 'ck_method_returnsqty',
       'ck_method_parametersqty', 'ck_method_methodsinvokedqty',
       'ck_method_assignme

In [6]:
df.columns=df.columns.str.replace(' ','_')

## Drop Code Smell

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.

In [7]:
df = df[df['agreement_experts_has_smell_name'] != 'none']
df_smell =df.drop('agreement_experts_has_smell_name', axis=1)
df['agreement_experts_has_smell_name'] = df['agreement_experts_has_smell_name'].astype(str)
smell = df['agreement_experts_has_smell_name']
df_smell=df_smell[['longmethod_label', 'featureenvy_label','ck_method_cbo','ck_class_cbo','ck_class_fanin']]
df_smell.columns

Index(['longmethod_label', 'featureenvy_label', 'ck_method_cbo',
       'ck_class_cbo', 'ck_class_fanin'],
      dtype='object')

#### Check the correlation and remove the high-correlated features from the dataset

In [8]:
corr = df_smell.corr(numeric_only = True)
threshold = 0.8
#columns = np.full((corr.shape[0],), True,dtype='object')
columns = np.isin(df_smell.columns,corr.index)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= threshold:
            if columns[j]:
                columns[j] = False
selected_columns = df_smell.columns[columns]
high_corr = set(df_smell.columns) - set(selected_columns)
df_smell = df_smell[selected_columns]
df_smell.columns

Index(['longmethod_label', 'featureenvy_label', 'ck_method_cbo',
       'ck_class_cbo', 'ck_class_fanin'],
      dtype='object')

##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

#### Setup the model creation with pycaret

In [9]:
s = setup(data=df_smell, target=smell, feature_selection=True,
          remove_multicollinearity=True,
          multicollinearity_threshold=0.85,
          feature_selection_method='classic',
          fix_imbalance=True, fold=10,index=False)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001505 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 578
[LightGBM] [Info] Number of data points in the train set: 9044, number of used features: 5
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,Description,Value
0,Session id,6077
1,Target,agreement_experts_has_smell_name
2,Target type,Multiclass
3,Target mapping,"blob: 0, dataclass: 1, featureenvy: 2, longmethod: 3"
4,Original data shape,"(9464, 6)"
5,Transformed data shape,"(11884, 2)"
6,Transformed train set shape,"(9044, 2)"
7,Transformed test set shape,"(2840, 2)"
8,Numeric features,5
9,Preprocess,True


In [10]:
df_smell = get_config('X')
df_smell

,longmethod_label,featureenvy_label,ck_method_cbo,ck_class_cbo,ck_class_fanin
0,0,0,7.0,4.0,4.0
1,0,0,34.0,33.0,40.0
2,0,0,0.0,0.0,0.0
3,0,0,0.0,0.0,0.0
4,0,0,47.0,12.0,11.0
...,...,...,...,...,...
9459,0,1,3.0,24.0,3.0
9460,0,1,0.0,0.0,0.0
9461,0,1,2.0,19.0,6.0
9462,0,0,0.0,0.0,0.0


##### Train the models. In this code the model was trained for the Long Method smell. From all models, we selected the top-5 in terms of F1.

In [11]:
top5_models = compare_models(n_select=5, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.3966,0.7064,0.3966,0.4926,0.4096,0.2146,0.2313,0.5840
rf,Random Forest Classifier,0.3942,0.7051,0.3942,0.4877,0.4068,0.2114,0.2285,0.8380
et,Extra Trees Classifier,0.3903,0.7053,0.3903,0.4904,0.4017,0.2092,0.2284,0.8150
gbc,Gradient Boosting Classifier,0.3904,0.0000,0.3904,0.4951,0.4003,0.2115,0.2313,1.6320
xgboost,Extreme Gradient Boosting,0.3890,0.6998,0.3890,0.4923,0.3993,0.2086,0.2292,0.6730
lightgbm,Light Gradient Boosting Machine,0.3877,0.7015,0.3877,0.4831,0.3975,0.2062,0.2241,2.4390
knn,K Neighbors Classifier,0.4034,0.6408,0.4034,0.3833,0.3534,0.1250,0.1389,0.4570
ada,Ada Boost Classifier,0.3516,0.0000,0.3516,0.4321,0.3446,0.1633,0.1848,0.6790
lr,Logistic Regression,0.3197,0.0000,0.3197,0.4056,0.3134,0.1086,0.1222,1.8930
lda,Linear Discriminant Analysis,0.3095,0.0000,0.3095,0.4176,0.2973,0.1080,0.1235,0.4120


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

##### Result by Smell

In [12]:
# Dicionário para armazenar os relatórios por modelo e por classe
reports = {}
# Lista para armazenar os dados
data = []
for model in top5_models:
    model_name = str(model).split('(')[0]  # Obtém o nome do modelo
    predictions = predict_model(model)  # Faz previsões nos dados de teste
    #print(predictions)

    # Gerar relatório por classe
    report = classification_report(predictions['agreement_experts_has_smell_name'], predictions['prediction_label'], output_dict=True)

    # Armazenar o relatório no dicionário
    reports[model_name] = report

# Exibir o relatório para cada tipo de smell e modelo
for model, report in reports.items():
    print(f"Classification Report for {model}:")
    for smell_class in ['blob', 'dataclass', 'featureenvy', 'longmethod']:
        if smell_class in report:
            print(f"\nSmell: {smell_class}")
            #print(f"  Classes: {report[smell_class]}")
            print(f"  Accuracy: {report['accuracy']:.2f}")
            print(f"  Precision: {report[smell_class]['precision']:.2f}")
            print(f"  Recall: {report[smell_class]['recall']:.2f}")
            print(f"  F1-score: {report[smell_class]['f1-score']:.2f}")
            data.append({
                "Model": model,
                "Smell": smell_class,
                "Accuracy": report["accuracy"],
                "Precision": report[smell_class]["precision"],
                "Recall": report[smell_class]["recall"],
                "F1-score": report[smell_class]["f1-score"]
            })
    print("=" * 80)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.3824,0.6997,0.3824,0.4751,0.3923,0.2009,0.2183


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.3824,0.6982,0.3824,0.4751,0.3923,0.2009,0.2183


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.3820,0.6997,0.3820,0.4745,0.3919,0.2004,0.2178


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.3842,0.7003,0.3842,0.4838,0.3930,0.2058,0.2247


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.3817,0.6966,0.3817,0.4800,0.3907,0.2027,0.2211


Classification Report for DecisionTreeClassifier:

Smell: blob
  Accuracy: 0.38
  Precision: 0.67
  Recall: 0.49
  F1-score: 0.57

Smell: dataclass
  Accuracy: 0.38
  Precision: 0.46
  Recall: 0.19
  F1-score: 0.27

Smell: featureenvy
  Accuracy: 0.38
  Precision: 0.36
  Recall: 0.38
  F1-score: 0.37

Smell: longmethod
  Accuracy: 0.38
  Precision: 0.20
  Recall: 0.57
  F1-score: 0.29
Classification Report for RandomForestClassifier:

Smell: blob
  Accuracy: 0.38
  Precision: 0.67
  Recall: 0.49
  F1-score: 0.57

Smell: dataclass
  Accuracy: 0.38
  Precision: 0.46
  Recall: 0.19
  F1-score: 0.27

Smell: featureenvy
  Accuracy: 0.38
  Precision: 0.36
  Recall: 0.38
  F1-score: 0.37

Smell: longmethod
  Accuracy: 0.38
  Precision: 0.20
  Recall: 0.57
  F1-score: 0.29
Classification Report for ExtraTreesClassifier:

Smell: blob
  Accuracy: 0.38
  Precision: 0.67
  Recall: 0.49
  F1-score: 0.57

Smell: dataclass
  Accuracy: 0.38
  Precision: 0.46
  Recall: 0.19
  F1-score: 0.27

Smell: fea

In [13]:
# Criar um DataFrame
df = pd.DataFrame(data)

# Salvar o DataFrame em um arquivo CSV
df.to_csv("smell_classification_report_FS3_Multilabel.csv", index=False)

In [14]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('multilabel_FS3.latex',index=False)
df3.to_csv('multilabel_FS3.csv',index=False)

#### Ensemble the top-5 models

In [16]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3952,0.7074,0.3952,0.4874,0.3959,0.2191,0.2396
1,0.3680,0.6974,0.3680,0.4515,0.3718,0.1839,0.2018
2,0.3846,0.7009,0.3846,0.4666,0.3935,0.1900,0.2042
3,0.3801,0.7124,0.3801,0.4744,0.3950,0.1894,0.2061
4,0.4063,0.7189,0.4063,0.5278,0.4139,0.2399,0.2701
5,0.3671,0.6871,0.3671,0.4649,0.3785,0.1844,0.2022
6,0.4063,0.7003,0.4063,0.5008,0.4215,0.2236,0.2407
7,0.4094,0.7129,0.4094,0.5215,0.4266,0.2328,0.2522
8,0.3897,0.7012,0.3897,0.4907,0.4052,0.2074,0.2264


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [17]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.3824,0.6982,0.3824,0.4751,0.3923,0.2009,0.2183


In [18]:
result = pull()

#### Plot the ensemble model

In [19]:
result.to_csv('models_multilabel_FS3.csv')

#### Finalize and save the ensemble model

In [20]:
final_best = finalize_model(blended_models)
save_model(final_best, 'models_multilabel_FS3')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['longmethod_label',
                                              'featureenvy_label',
                                              'ck_method_cbo', 'ck_class_cbo',
                                              'ck_class_fanin'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=T...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',
                                         max_depth=None, max_features='sqrt',
                  

##### Drop the code smells that are not necessary for the model. For instance, this code is set up to the Long Method smell. To train other models, remove the other code smells.